### **Step 1: Setup environment**

In [ ]:
# Install necessary libraries
!pip install lpips scikit-image

# Import required libraries
import os
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import lpips
import pandas as pd
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

print("Environment setup completed.")

Environment setup completed.


**lpips:** Imported directly, this library is used for evaluating perceptual image patch similarity, which differs from traditional metrics like PSNR and SSIM by focusing on perceptual differences rather than pixel-level differences.

**scikit-image:** A collection of algorithms for image processing in Python. It's widely used for tasks such as image segmentation, geometric transformations, color space manipulation, analysis, filtering, morphology, feature detection, and more.

**os:** A module for interacting with the operating system, useful for file path operations, directory navigation, and more.

**numpy:** A fundamental package for scientific computing in Python, providing support for large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.

**peak_signal_noise_ratio (PSNR):** A metric used to measure the quality of reconstruction of lossy compression codecs. It's commonly used in image processing to compare the quality of an output image to its original.

**structural_similarity (SSIM):** A method for measuring the similarity between two images. It's often used to assess the quality of images after compression or other types of processing.

**pandas:** A library providing high-performance, easy-to-use data structures, and data analysis tools.

**from scipy.stats import spearmanr:** Imports a function for calculating the Spearman rank-order correlation coefficient, which measures the monotonicity of the relationship between two datasets.

**matplotlib.pyplot:** A plotting library that provides a MATLAB-like interface for making plots and visualizations in Python.

### **Step 2: Download and extract the dataset**

In [ ]:
# Download the dataset
!wget http://ece.iisc.ac.in/~rajivs/courses/aip2016/hw5.rar

# Extract the contents of the downloaded archive
!unrar x hw5.rar

print("Dataset downloaded and extracted.")

# Extract gblur.rar and refimgs.rar
!unrar x hw5/gblur.rar
!unrar x hw5/refimgs.rar

print("Folders extracted:")
for folder in ["gblur", "refimgs"]:
    folder_path = folder
    if os.path.exists(folder_path):
        print(f"- {folder}: {len(os.listdir(folder_path))} files")
    else:
        print(f"- {folder}: Folder not found")

--2024-04-01 14:17:45--  http://ece.iisc.ac.in/~rajivs/courses/aip2016/hw5.rar
Resolving ece.iisc.ac.in (ece.iisc.ac.in)... 13.71.53.212
Connecting to ece.iisc.ac.in (ece.iisc.ac.in)|13.71.53.212|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ece.iisc.ac.in/~rajivs/courses/aip2016/hw5.rar [following]
--2024-04-01 14:17:45--  https://ece.iisc.ac.in/~rajivs/courses/aip2016/hw5.rar
Connecting to ece.iisc.ac.in (ece.iisc.ac.in)|13.71.53.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85354498 (81M)
Saving to: ‘hw5.rar.8’

hw5.rar.8           100%[===================>]  81.40M  12.6MB/s    in 7.7s    

2024-04-01 14:17:54 (10.5 MB/s) - ‘hw5.rar.8’ saved [85354498/85354498]


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from hw5.rar


Would you like to replace the existing file hw5/gblur.rar
69342543 bytes, modified on 2016-03-23 18:09
with a new one
69342543 bytes, modif

This section of the code is crucial for ensuring that the necessary data for the project is available and organized. By downloading and extracting specific parts of the dataset, it prepares the ground for subsequent image processing or analysis tasks.

### **Step 3: Implement the perceptual quality metrics**

In [ ]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import lpips

def calculate_psnr(img1, img2):
    return peak_signal_noise_ratio(img1, img2)

def calculate_ssim(img1, img2):
    return structural_similarity(img1, img2, channel_axis=2)

def calculate_lpips_vgg(img1, img2):
    loss_fn = lpips.LPIPS(net='vgg')
    img1_tensor = lpips.im2tensor(img1)
    img2_tensor = lpips.im2tensor(img2)
    return loss_fn(img1_tensor, img2_tensor).item()

def calculate_lpips_alexnet(img1, img2):
    loss_fn = lpips.LPIPS(net='alex')
    img1_tensor = lpips.im2tensor(img1)
    img2_tensor = lpips.im2tensor(img2)
    return loss_fn(img1_tensor, img2_tensor).item()

print("Perceptual quality metrics implemented.")

Perceptual quality metrics implemented.


**calculate_psnr(img1, img2)**
PSNR (Peak Signal-to-Noise Ratio) is a widely used measure to assess the quality of reconstruction of lossy compression codecs. The higher the PSNR, the better the quality of the compressed or reconstructed image. It is expressed in logarithmic decibels (dB) scale.

**calculate_ssim(img1, img2)**
SSIM (Structural Similarity Index) is a method for measuring the similarity between two images. The SSIM index is a decimal value between -1 and 1; 1 indicates perfect similarity. It considers changes in structural information, luminance, and contrast, providing a more accurate measure of perceptual similarity than PSNR.
Usage: This function calculates the SSIM of two images, specifying that the comparison should be made color channel-wise (channel_axis=2) for color images.

**calculate_lpips_vgg(img1, img2): Uses a VGG network.
calculate_lpips_alexnet(img1, img2): Uses an AlexNet network.**
LPIPS  (Learned Perceptual Image Patch Similarity) measures the perceptual similarity between two images using deep networks. Unlike PSNR and SSIM, LPIPS uses learned features from deep networks (VGG, AlexNet) to assess perceptual similarity, which can align more closely with human judgment. It's particularly useful for tasks where fine-grained perceptual differences are critical, such as in image generation or super-resolution.
Usage: Both functions convert input images to tensors, then compute the LPIPS score using the specified network (VGG or AlexNet). This approach provides flexibility in choosing the underlying neural network model based on the specific requirements or expected characteristics of the images being compared.

### **Step 4: Load and preprocess the data**

In [ ]:
import os
import numpy as np
from PIL import Image
import scipy.io

# Load reference image names
ref_names = scipy.io.loadmat('hw5/hw5.mat')['refnames_blur'][0]

# Load human opinion scores
dmos_scores = scipy.io.loadmat('hw5/hw5.mat')['blur_dmos'][0]

# Load original image indicators
is_original = scipy.io.loadmat('hw5/hw5.mat')['blur_orgs'][0]

# Initialize lists to store image paths and scores
distorted_imgs = []
reference_imgs = []
scores = []

# Iterate over distorted images
for i, img_name in enumerate(os.listdir('gblur')):
    if img_name.endswith('.bmp'):
        # Load distorted image
        distorted_img_path = os.path.join('gblur', img_name)
        distorted_imgs.append(distorted_img_path)

        # Find corresponding reference image
        if i < len(ref_names):
            ref_name = ref_names[i][0]
            ref_img_path = os.path.join('refimgs', ref_name)
            reference_imgs.append(ref_img_path)
        else:
            reference_imgs.append(None)

        # Store score if not an original image
        if i < len(is_original) and not is_original[i]:
            scores.append(dmos_scores[i])
        else:
            scores.append(None)

print(f"Number of distorted images: {len(distorted_imgs)}")
print(f"Number of reference images: {len([img for img in reference_imgs if img is not None])}")
print(f"Number of valid scores: {len([s for s in scores if s is not None])}")

Number of distorted images: 174
Number of reference images: 172
Number of valid scores: 143


The code loads three pieces of information from a .mat file:


*   ref_names: Names of reference images corresponding to the distorted images.
*   dmos_scores: Differential Mean Opinion Scores (DMOS), which are human-assigned scores indicating the perceived quality of the distorted images relative to their references. Lower scores typically mean better perceived quality.
*   is_original: A binary indicator showing whether an image is an original (reference) image.




The code then initializes lists to store paths to distorted images, reference images, and DMOS scores. It iterates over the distorted images stored in a directory named gblur, performing the following steps for each image:


*   Distorted Image Processing: Adds the path of the distorted image to the distorted_imgs list.
*   Reference Image Matching: Attempts to match each distorted image with its reference image using the ref_names list. If a match is found within the bounds of ref_names, it adds the path to the reference_imgs list. Otherwise, it appends None.
*   DMOS Score Association: Associates a DMOS score with each distorted image if the image is not marked as an original in is_original. If the image is an original or if it exceeds the bounds of the is_original list, it appends None.

### **Step 5: Compute quality scores**

In [ ]:
def calculate_psnr(img1, img2):
    data_range = np.max(img2) - np.min(img2)
    return peak_signal_noise_ratio(img1, img2, data_range=data_range)

In [ ]:
from tqdm import tqdm
from skimage.transform import resize

# Initialize LPIPS models
lpips_vgg = lpips.LPIPS(net='vgg')
lpips_alexnet = lpips.LPIPS(net='alex')

# Initialize lists to store quality scores
psnr_scores = []
ssim_scores = []
lpips_vgg_scores = []
lpips_alexnet_scores = []

# Iterate over distorted images and calculate quality scores
for i, (distorted_img_path, ref_img_path) in enumerate(tqdm(zip(distorted_imgs, reference_imgs), total=len(distorted_imgs))):
    if ref_img_path is not None:
        # Load distorted and reference images
        distorted_img = np.array(Image.open(distorted_img_path).convert('RGB'))
        ref_img = np.array(Image.open(ref_img_path).convert('RGB'))

        # Resize distorted image to match reference image dimensions
        distorted_img_resized = resize(distorted_img, ref_img.shape[:2], anti_aliasing=True, preserve_range=True)

        # Calculate quality scores
        psnr = calculate_psnr(distorted_img_resized, ref_img)
        ssim = calculate_ssim(distorted_img_resized, ref_img)
        lpips_vgg_score = lpips_vgg(lpips.im2tensor(distorted_img_resized), lpips.im2tensor(ref_img)).item()
        lpips_alexnet_score = lpips_alexnet(lpips.im2tensor(distorted_img_resized), lpips.im2tensor(ref_img)).item()

        psnr_scores.append(psnr)
        ssim_scores.append(ssim)
        lpips_vgg_scores.append(lpips_vgg_score)
        lpips_alexnet_scores.append(lpips_alexnet_score)
    else:
        psnr_scores.append(None)
        ssim_scores.append(None)
        lpips_vgg_scores.append(None)
        lpips_alexnet_scores.append(None)

print("Quality scores computed.")

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/vgg.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 174/174 [24:43<00:00,  8.52s/it]

Quality scores computed.


Four lists are initialized to store the quality scores calculated for each image pair:


*   psnr_scores
*   ssim_scores
*   lpips_vgg_scores
*   lpips_alexnet_scores



**Image Loading and Resizing:** Both the distorted and reference images are loaded and converted to RGB. The distorted image is resized to match the reference image's dimensions to ensure comparability.

Quality Metric Calculation:


*   **PSNR:** The Peak Signal-to-Noise Ratio is calculated between the resized distorted image and the reference image. The data_range is dynamically determined based on the reference image's pixel value range to improve accuracy.
*   **SSIM:** The Structural Similarity Index is calculated to measure the perceptual similarity between the two images.
*   **LPIPS Scores:** The Learned Perceptual Image Patch Similarity scores are computed using both the VGG and AlexNet models. These scores are derived from deep learning models and aim to provide a measure of perceptual similarity that more closely aligns with human visual perception.



### **Step 6: Prepare the data for analysis**

In [ ]:
import pandas as pd

# Create a dataframe to store the quality scores
df = pd.DataFrame({
    'Distorted Image': distorted_imgs,
    'Reference Image': reference_imgs,
    'DMOS Score': scores,
    'PSNR': psnr_scores,
    'SSIM': ssim_scores,
    'LPIPS (VGG)': lpips_vgg_scores,
    'LPIPS (AlexNet)': lpips_alexnet_scores
})

# Remove rows with missing scores
df_valid = df.dropna(subset=['DMOS Score'])

print(f"Number of valid rows: {len(df_valid)}")
print(f"Number of removed rows: {len(df) - len(df_valid)}")

# Display the first few rows of the dataframe
print("\nDataframe head:")
print(df_valid.head())

Number of valid rows: 143
Number of removed rows: 31

Dataframe head:
    Distorted Image               Reference Image  DMOS Score       PSNR  \
0  gblur/img156.bmp              refimgs/caps.bmp   42.127765   9.793991   
1   gblur/img98.bmp  refimgs/churchandcapitol.bmp   37.462720   7.582538   
2   gblur/img23.bmp           refimgs/monarch.bmp   43.264560  11.130536   
3   gblur/img40.bmp        refimgs/lighthouse.bmp   19.510097  11.426720   
4   gblur/img93.bmp             refimgs/plane.bmp   72.829422   7.305499   

       SSIM  LPIPS (VGG)  LPIPS (AlexNet)  
0 -0.000998     0.833526         0.842938  
1 -0.000390     0.763065         0.810280  
2 -0.003895     0.853667         0.855812  
3  0.001432     0.804289         0.889132  
4  0.019894     0.752861         0.900414  


**pd.DataFrame({...}):** A pandas DataFrame is created with columns labeled 'Distorted Image', 'Reference Image', 'DMOS Score', 'PSNR', 'SSIM', 'LPIPS (VGG)', and 'LPIPS (AlexNet)'. Each column is filled with the corresponding data from the lists populated in the previous steps. This structure allows for easy visualization and manipulation of the dataset, linking each distorted image to its reference, the human-assigned DMOS score, and the computed quality metrics.

**df.dropna(subset=['DMOS Score']):** This line removes any rows from the DataFrame that have a missing ('NaN') value in the 'DMOS Score' column. The assumption here is that without a DMOS score, the row lacks the essential human opinion score needed for certain types of analysis, such as comparing human perception against computed quality metrics. The resulting DataFrame is stored in df_valid.

### **Step 7: Compute correlation coefficients**

In [ ]:
from scipy.stats import spearmanr

# Compute Spearman rank order correlation coefficients
spearman_psnr = spearmanr(df_valid['DMOS Score'], df_valid['PSNR'])[0]
spearman_ssim = spearmanr(df_valid['DMOS Score'], df_valid['SSIM'])[0]
spearman_lpips_vgg = spearmanr(df_valid['DMOS Score'], df_valid['LPIPS (VGG)'])[0]
spearman_lpips_alexnet = spearmanr(df_valid['DMOS Score'], df_valid['LPIPS (AlexNet)'])[0]

print("Spearman rank order correlation coefficients:")
print(f"PSNR: {spearman_psnr:.4f}")
print(f"SSIM: {spearman_ssim:.4f}")
print(f"LPIPS (VGG): {spearman_lpips_vgg:.4f}")
print(f"LPIPS (AlexNet): {spearman_lpips_alexnet:.4f}")

Spearman rank order correlation coefficients:
PSNR: -0.0509
SSIM: -0.0386
LPIPS (VGG): 0.0407
LPIPS (AlexNet): 0.0692


Based on the computed Spearman rank order correlation coefficients, we can make the following observations:

- **PSNR:**
    - The correlation coefficient between PSNR and DMOS scores is $-0.0509$, which indicates a weak negative correlation.
    - This suggests that PSNR may not be a strong indicator of human perception of image quality for this dataset.
- **SSIM:**
    - The correlation coefficient between SSIM and DMOS scores is $-0.0386$, which also indicates a weak negative correlation.
    - Similar to PSNR, SSIM does not seem to correlate well with human perception of image quality for this dataset.
- **LPIPS (VGG):**
    - The correlation coefficient between LPIPS (VGG) and DMOS scores is $0.0407$, indicating a weak positive correlation.
    - While the correlation is slightly stronger than PSNR and SSIM, it is still relatively weak.
- **LPIPS (AlexNet):**
    - The correlation coefficient between LPIPS (AlexNet) and DMOS scores is $0.0692$, which is the highest among the evaluated metrics.
    - Although the correlation is still relatively weak, LPIPS (AlexNet) appears to have the strongest correlation with human perception of image quality among the considered metrics.

**Comparing the performance of the quality metrics:**

- LPIPS (AlexNet) has the highest absolute correlation coefficient ($0.0692$), indicating that it has the strongest correlation with human perception among the evaluated metrics.
- LPIPS (VGG) follows LPIPS (AlexNet) with a correlation coefficient of $0.0407$.
- PSNR and SSIM have negative correlation coefficients, suggesting that they may not be suitable for assessing image quality in this particular dataset.
